In [3]:
import pandas as pd
import pyodbc
import os

# Año de interés definido aquí
año_interes = 2024

# Información de conexión
server = 'ec0038app05'
database = 'SHAYA'
username = 'python_user'
password = 'python_user'
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Función para obtener los DataFrames
def obtener_dataframe(query, conn):
    return pd.read_sql(query, conn)

conn = pyodbc.connect(conn_str)

# Consulta SQL
queryO = f""" 
SELECT ITEM_NAME, START_WO, END_WO, PLAN_TYPE_TEXT, WO_NUMBER, VERIFIED
FROM VT_WELLJOBLOG_en_US
WHERE YEAR(END_WO) = {año_interes} AND PLAN_TYPE_TEXT = 'Opex'
"""

dfO = obtener_dataframe(queryO, conn)

conn.close()

print(dfO.head())

C:\Users\aperez143\AppData\Local\Temp\ipykernel_6276\3703258912.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


    ITEM_NAME            START_WO              END_WO PLAN_TYPE_TEXT  \
0    CNOF-056 2024-01-20 06:00:00 2024-01-25 20:00:00           Opex   
1  CLBA-007R1 2024-01-13 15:00:00 2024-01-23 02:00:00           Opex   
2   CLBC-048H 2024-01-02 13:00:00 2024-01-09 13:00:00           Opex   
3    ACAM-146 2024-01-22 17:00:00 2024-01-31 19:00:00           Opex   
4   CLBA-036I 2024-01-05 20:00:00 2024-01-27 00:00:00           Opex   

   WO_NUMBER VERIFIED  
0        8.0     True  
1        6.0     True  
2        1.0     True  
3        3.0     None  
4        1.0     True  


In [ ]:
import pandas as pd
import pyodbc
import os

# Año de interés definido aquí
año_interes = 2024

# Información de conexión
server = 'ec0038app05'
database = 'SHAYA'
username = 'python_user'
password = 'python_user'
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

# Función para obtener los DataFrames
def obtener_dataframe(query, conn):
    return pd.read_sql(query, conn)

# Función para filtrar el DataFrame por trimestre y WO_NUMBER
def filtrar_por_trimestre(df, meses):
    return df[df['END_WO'].dt.month.isin(meses) & df['WO_NUMBER'].notna() & (df['WO_NUMBER'] != 0)].copy()

# Función para filtrar el DataFrame por trimestre para el caso de CPI
def filtrar_por_trimestreCo(df, meses):
    return df[df['END_WO'].dt.month.isin(meses)].copy()

# Función para crear directorios
def crear_directorios(df, ruta_trimestre, tipo_actividad):
    os.makedirs(ruta_trimestre, exist_ok=True)

    if tipo_actividad == 'CPI':
        ruta_completacion = os.path.join(ruta_trimestre, 'COMPLETACION')
        os.makedirs(ruta_completacion, exist_ok=True)
        for _, fila in df.iterrows():
            pozo = fila['ITEM_NAME']
            nombre_carpeta_pozo = f'{pozo}'
            ruta_carpeta_pozo = os.path.join(ruta_completacion, nombre_carpeta_pozo)
            os.makedirs(ruta_carpeta_pozo, exist_ok=True)

            # Crear subcarpetas dentro de la carpeta del pozo
            subcarpetas = ['1.1 Prognosis y Programas', '1.2 Soporte para inicio de Trabajos', 
                       '1.3 Reportes diarios de ejecución', '1.4 Sumarios Finales CPI', 
                       '1.5 Actas de Inicio y Recepción']
            for subcarpeta in subcarpetas:
                os.makedirs(os.path.join(ruta_carpeta_pozo, subcarpeta), exist_ok=True)


    elif tipo_actividad == 'OPEX':
        for _, fila in df.iterrows():
            pozo = fila['ITEM_NAME']
            wo_number = int(fila['WO_NUMBER'])
            nombre_carpeta_pozo = f'{pozo} WO {wo_number}'
            ruta_carpeta_pozo = os.path.join(ruta_trimestre, nombre_carpeta_pozo)
            os.makedirs(ruta_carpeta_pozo, exist_ok=True)

            # Crear subcarpetas dentro de la carpeta del pozo
            subcarpetas = ['3.1 Prognosis y Programas', '3.2 Soporte para inicio de Trabajos', 
                       '3.3 Reportes diarios de ejecución', '3.4 Diagramas Mecánicos', 
                       '3.5 Registros', '3.6 Sumarios Finales', '3.7 Post Mortem Pozos', 
                       '3.8 Actas de Inicio y Recepción']
            for subcarpeta in subcarpetas:
                os.makedirs(os.path.join(ruta_carpeta_pozo, subcarpeta), exist_ok=True)
    
    elif tipo_actividad == 'CAPEX':
        for _, fila in df.iterrows():
            pozo = fila['ITEM_NAME']
            wo_number = int(fila['WO_NUMBER'])
            nombre_carpeta_pozo = f'{pozo} WO {wo_number}'
            ruta_carpeta_pozo = os.path.join(ruta_trimestre, nombre_carpeta_pozo)
            os.makedirs(ruta_carpeta_pozo, exist_ok=True)

            # Crear subcarpetas dentro de la carpeta del pozo
            subcarpetas = ['2.1 Prognosis y Programas', '2.2 Soporte para inicio de Trabajos', 
                       '2.3 Reportes diarios de ejecución',  '2.4 Sumarios Finales WO', 
                       '2.5 Actas de Inicio y Recepción']
            for subcarpeta in subcarpetas:
                os.makedirs(os.path.join(ruta_carpeta_pozo, subcarpeta), exist_ok=True)

# Conexión a la base de datos y obtención de los DataFrames
conn = pyodbc.connect(conn_str)

# Consulta SQL
queryO = f""" 
SELECT ITEM_NAME, START_WO, END_WO, PLAN_TYPE_TEXT, WO_NUMBER
FROM VT_WELLJOBLOG_en_US
WHERE YEAR(END_WO) = {año_interes} AND PLAN_TYPE_TEXT = 'Opex' AND VERIFIED = 'True'
"""

queryC = f""" 
SELECT ITEM_NAME, START_WO, END_WO, PLAN_TYPE_TEXT, WO_NUMBER
FROM VT_WELLJOBLOG_en_US
WHERE YEAR(END_WO) = {año_interes} AND PLAN_TYPE_TEXT = 'Capex' AND VERIFIED = 'True'
"""

queryCompletacion = f"""
SELECT ITEM_NAME, START_WO, END_WO, WELL_STATUS_TEXT, WO_NUMBER
FROM VT_WELLJOBLOG_en_US
WHERE YEAR(END_WO) = {año_interes} AND WELL_STATUS_TEXT = 'CPI' AND VERIFIED = 'True'
"""

dfO = obtener_dataframe(queryO, conn)
dfC = obtener_dataframe(queryC, conn)
dfCo = obtener_dataframe(queryCompletacion, conn)
conn.close()

# Directorio base
directorio_base = rf'\\dir.slb.com\NSA\SAM_Collaborate\EC0037\90_Public\Tests\01 INFORMES TRIMESTRALES'
ruta_año = os.path.join(directorio_base, f'Reporte Trimestral {año_interes}')

# Filtrar DataFrames por trimestre
trimestres = [(1, 2, 3), (4, 5, 6), (7, 8, 9), (10, 11, 12)]
nombres_trimestres = ['01. Reporte Trimestral #1 2023', '02. Reporte Trimestral #2 2023', 
                      '03. Reporte Trimestral #3 2023', '04. Reporte Trimestral #4 2023']

for i, meses in enumerate(trimestres):
    dfO_trimestre = filtrar_por_trimestre(dfO, meses)
    dfC_trimestre = filtrar_por_trimestre(dfC, meses)
    dfCo_trimestre = filtrar_por_trimestreCo(dfCo, meses)
    os.makedirs(os.path.join(ruta_año, nombres_trimestres[i], '4. Apoyo a la Operación'), exist_ok=True)
    ruta_trimestre_CPI = os.path.join(ruta_año, nombres_trimestres[i], '1. Perforación & Completación de Pozos Nuevos')
    ruta_trimestre_Opex = os.path.join(ruta_año, nombres_trimestres[i], '3. Intervenciones con Torre (Pulling)')
    ruta_trimestre_Capex = os.path.join(ruta_año, nombres_trimestres[i], '2. Reacondicionamiento de Pozos (Workover CAPEX)')
    crear_directorios(dfCo_trimestre, ruta_trimestre_CPI, 'CPI')
    crear_directorios(dfO_trimestre, ruta_trimestre_Opex, 'OPEX')
    crear_directorios(dfC_trimestre, ruta_trimestre_Capex, 'CAPEX')